In [80]:
from __future__ import print_function

import time
import requests
from bs4 import BeautifulSoup
import pymongo
import pandas as pd
import numpy as np
import re
import unicodedata

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wnlzer = WordNetLemmatizer()

from pprint import pprint
from time import time
import logging

import matplotlib.pyplot as plt

source_path = 'test_data'
#source_file = 'test_strains_reviews.csv'
target_path = '/Users/jordanweil/green_rex/test_data'

In [2]:
def list_o_strains(i):
    LOS = []
    Type = []
    r = requests.get(i)
    soup2 = BeautifulSoup(r.content, 'html.parser')
    strains = soup2.find_all('a', class_="ga_Explore_Strain_Tile")
    for s in strains:
        LOS.append(str(s.attrs['href'])[1:])
    return LOS

In [3]:
def get_reviews2(l):
    """Pass in a list of URL's and return them in a mongo db table as a dicitonary with 
    {'url', 'html'} and their corresponding values"""
    r = requests.get(l)
    html = (r.content)
    return html

In [84]:
def parse_docs2(d):
    """Parse the HTML docs that we have stored in a dictionary, return as a list.
    Also scrape and parse star rating for each review """
    strain_description = []
    strain_flavors = []
    strain_effects = []
    effects = []
    negs = []
    meds = []
    
    soup = BeautifulSoup(d, 'html.parser')
    desc = soup.find_all('div', itemprop='description') 
    for d in desc:
        text = d.text
        remove_n = re.sub('\n' , " " ,text )
        norm_text = unicodedata.normalize("NFKD", remove_n)
        """token = remove_punch.lower().split()
        srm_token = [wnlzer.lemmatize(i) for i in token if not i in set(stopwords.words('english'))]
        clean_text = " ".join(srm_token)"""
        strain_description.append(norm_text)
    
    #create list of all the effects
    effect = soup.find_all('div', attrs={'class':'histogram-label'})
    for e in effect:
        strain_effects.append(e.text)
        
    # create smaller lists of good, bad, medical effects
    for i in strain_effects[:5]:
        effects.append(i)
    for i in strain_effects[10:]:
        negs.append(i)
    for i in strain_effects[5:10]:
        meds.append(i)
    
    #Create list of each strain flavor
    flavors = soup.find_all('div', class_='flavor-name')
    for f in flavors:
        strain_flavors.append(f.text[3:])
        
    return strain_description, strain_flavors, effects, meds, negs

In [58]:
def strain_dict_entry(strain, desc, flavors, effects, meds, negs, straindict):
    if straindict is None:
        straindict = {}
        
    if strain not in straindict:
        straindict[strain] = {
            "desc" : desc,
            "flavors" : flavors,
            'positive': effects,
            'medical': meds,
            'negative': negs
        }
    return straindict

In [59]:
def scraper_dummy1(los):
    """pass in a list of strains to be scraped from Leafly.
    returns dictionary keyed by strains w/strain info(reviews) as values"""
    cnt = 0
    strain_d = None
    for s in los:
        strain = s
        url = "https://www.leafly.com/{}".format(strain)
        d = get_reviews2(url)
        desc, flavors, effects, meds, negs = parse_docs2(d)
        if len(desc) == 0:
            break
     
    for d, f, e, m, n in zip(desc, flavors, effects, meds, negs):
            strain_d = strain_dict_entry(strain, d, f, e, m, n, strain_d)
    if cnt % 10 == 0:
                print(cnt)
    cnt +=1
            
    return strain_d 

In [60]:
los = list_o_strains('https://www.leafly.com/explore')
los

['hybrid/blue-dream',
 'sativa/sour-diesel',
 'hybrid/gsc',
 'sativa/green-crack',
 'hybrid/og-kush',
 'indica/granddaddy-purple',
 'hybrid/original-glue',
 'hybrid/white-widow',
 'hybrid/blue-dream',
 'sativa/sour-diesel',
 'hybrid/gsc',
 'sativa/green-crack',
 'hybrid/og-kush',
 'indica/granddaddy-purple',
 'hybrid/original-glue',
 'hybrid/white-widow',
 'sativa/jack-herer',
 'indica/bubba-kush',
 'hybrid/pineapple-express',
 'hybrid/trainwreck',
 'hybrid/ak-47',
 'sativa/durban-poison',
 'indica/northern-lights',
 'hybrid/headband',
 'indica/blue-cheese',
 'sativa/strawberry-cough',
 'hybrid/chemdawg',
 'indica/purple-kush',
 'sativa/lemon-haze',
 'sativa/super-lemon-haze',
 'indica/grape-ape',
 'indica/blueberry',
 'sativa/alaskan-thunder-fuck',
 'sativa/super-silver-haze',
 'indica/blackberry-kush',
 'hybrid/cherry-pie',
 'indica/master-kush',
 'hybrid/skywalker-og',
 'hybrid/cheese',
 'indica/death-star',
 'sativa/chocolope',
 'sativa/amnesia-haze',
 'hybrid/tahoe-og',
 'sativa/m

In [61]:
sd = scraper_dummy1(los)

KeyboardInterrupt: 

In [ ]:
parse_docs2('https://www.leafly.com/')

In [55]:
l = list_o_strains('https://www.leafly.com/hybrid/blue-dream')

In [70]:
def scraper_dummy1(los):
    """pass in a list of strains to be scraped from Leafly.
    returns dictionary keyed by strains w/strain info(reviews) as values"""
    cnt = 0
    strain_d = None
    for s in los:
        strain = s
        url = "https://www.leafly.com/{}".format(strain)
        d = get_reviews2(url)
        desc, flavors, effects, meds, negs = parse_docs2(d)
        if len(desc) == 0:
            break
            
        strain_d = strain_dict_entry(strain, desc, flavors, effects, meds, negs, strain_d)
        if cnt % 10 == 0:
            print(cnt)
        cnt +=1
            
    return strain_d 

In [87]:
d_new = scraper_dummy1(los)

0
10
20
30
40
50


In [88]:
d_new

{'hybrid/blue-dream': {'desc': ['Blue Dream, a sativa-dominant hybrid originating in California, has achieved legendary status among West Coast strains. Crossing a Blueberry indica with the sativa Haze, Blue Dream balances full-body relaxation with gentle cerebral invigoration. Novice and veteran consumers alike enjoy the level effects of Blue Dream, which ease you gently into a calm euphoria. Some Blue Dream phenotypes express a more indica-like look and feel, but the sativa-leaning variety remains most prevalent. With a sweet berry aroma redolent of its Blueberry parent, Blue Dream delivers swift symptom relief without heavy sedative effects. This makes Blue Dream a popular daytime medicine for patients treating pain, depression, nausea, and other ailments requiring a high THC strain.  '],
  'flavors': ['Blueberry', 'Berry', 'Sweet'],
  'positive': ['Happy', 'Relaxed', 'Euphoric', 'Uplifted', 'Creative'],
  'medical': ['Stress', 'Depression', 'Pain', 'Headaches', 'Fatigue'],
  'negat

In [91]:
d_new.keys()

dict_keys(['hybrid/blue-dream', 'sativa/sour-diesel', 'hybrid/gsc', 'sativa/green-crack', 'hybrid/og-kush', 'indica/granddaddy-purple', 'hybrid/original-glue', 'hybrid/white-widow', 'sativa/jack-herer', 'indica/bubba-kush', 'hybrid/pineapple-express', 'hybrid/trainwreck', 'hybrid/ak-47', 'sativa/durban-poison', 'indica/northern-lights', 'hybrid/headband', 'indica/blue-cheese', 'sativa/strawberry-cough', 'hybrid/chemdawg', 'indica/purple-kush', 'sativa/lemon-haze', 'sativa/super-lemon-haze', 'indica/grape-ape', 'indica/blueberry', 'sativa/alaskan-thunder-fuck', 'sativa/super-silver-haze', 'indica/blackberry-kush', 'hybrid/cherry-pie', 'indica/master-kush', 'hybrid/skywalker-og', 'hybrid/cheese', 'indica/death-star', 'sativa/chocolope', 'sativa/amnesia-haze', 'hybrid/tahoe-og', 'sativa/maui-wowie', 'hybrid/platinum-gsc', 'sativa/harlequin', 'indica/gods-gift', 'indica/la-confidential', 'hybrid/agent-orange', 'indica/purple-urkle', 'hybrid/lemon-kush', 'hybrid/mazar-x-blueberry', 'hybrid/